In [ ]:
import torch
from meas_get_data import *
from meas_NN_classes import *
from meas_dataset import *
import os 
from tqdm import tqdm
import logging
from meas_test_func_fs import *
from meas_dataloader import *
from meas_train_funcs import *
from model_params import get_model_params
torch.set_default_dtype(torch.float64)


In [ ]:
parameter_set = get_model_params(testing_mode=False)
model_paths = {
    "OR_LSTM": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_OR_LSTM_expnumb_640.pth",
    "OR_MLP": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_OR_MLP_expnumb_123.pth",
    "OR_TCN": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_OR_TCN_expnumb_644.pth",
    "LSTM": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_LSTM_expnumb_564.pth",
    "MLP": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_MLP_expnumb_367.pth",
    "TCN": r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\Trained_networks\modeltype_TCN_expnumb_150.pth"}


if os.name == "nt":
    path_test_data=r"C:\Users\StrasserP\Documents\NN_Paper\Code_4_paper\messdaten\TEST-Messdaten_30traj_7times500steps.csv"
else:
    path_train_data=r"/home/rdpusr/Documents/Code_4_paper/messdaten/TEST-Messdaten_30traj_7times500steps.csv"
test_data = get_data(path_test_data,num_inits=0)

for parameters in parameter_set:

    if not os.path.exists(model_paths[parameters["model_flag"]]):
                continue    

    if "LSTM" in parameters["model_flag"]:
        model = OR_LSTM(input_size=4, hidden_size=parameters["h_size"], out_size=2, 
                        layers=parameters["l_num"], window_size=parameters["window_size"], flag=parameters["model_flag"]).to(device)

    if "MLP" in parameters["model_flag"]:

        model = OR_MLP(input_size=4*parameters["window_size"], hidden_size=parameters["h_size"],
                        output_size=2, l_num=parameters["l_num"], window_size=parameters["window_size"], flag=parameters["model_flag"]).to(device)

    if "TCN" in parameters["model_flag"]:
        num_channels = [parameters["n_hidden"]] * parameters["levels"]
        model = OR_TCN(input_size=4, output_size=2, num_channels=num_channels,
                        kernel_size=parameters["kernel_size"], dropout=parameters["dropout"], windowsize=parameters["window_size"], flag=parameters["model_flag"]).to(device)

    model.load_state_dict(torch.load(model_paths[model.get_flag()], map_location=device))

    if parameters["model_flag"] != model.get_flag():
        print("Parameter list model flag does not match the model flag used!")
    print(model.get_flag())
    test_error = test(data=test_data, model=model, window_size=parameters["window_size"], specific_index=26, display_plots=1)
